# Manage connections and discover assets

This notebook contains steps and code to demonstrate support for connections in Cloud Pak for Data using
[`cpdctl`](https://github.com/IBM/cpdctl) CLI tool. 

Some familiarity with Python is helpful. This notebook uses Python 3.7.


## Table of contents

#### 1. Installation and Configuration of CPDCTL

    1.1 Install the latest version of cpdctl 
    1.2 Check cpdctl version 
    1.3 Add CPD cluster configuration 
    
    
#### 2. Demo cpdctl connection commands

    2.0 Create a project for connection or Choose an existing project
    2.1 Create a connection
    2.2 List connections
    2.3 Discover connection ad-hoc
    2.4 Discover data asset (not available until CPD 3.5)
    2.5 Get connection
    2.6 Update connection
    2.7 List actions
    2.8 Perform action
    2.9 Discover connection
    2.10 Delete connection
    2.11 Clean up - Delete project created at the beggining (containing our connection)
    
#### 3. Demo cpdctl connection datasource types commands

    3.1 List datasource types
    3.2 Get datasource type
    
### Another useful documentation regarding connections parameters can be found here:

Connections REST API docs https://api.dataplatform.dev.cloud.ibm.com/v2/connections/api/explorer/

Watson Data API docs https://cloud.ibm.com/apidocs/watson-data-api#listconnections



In [1]:
import base64
import json
import os
import requests
import platform
import tarfile
import zipfile
from IPython.core.display import display, HTML

## 1. Installation and Configuration of CPDCTL


### 1.1 Install the latest version of cpdctl

In [2]:
PLATFORM = platform.system().lower()
CPDCTL_ARCH = "{}_amd64".format(PLATFORM)
CPDCTL_RELEASES_URL="https://api.github.com/repos/IBM/cpdctl/releases"
CWD = os.getcwd()
PATH = os.environ['PATH']
CPD_CONFIG = os.path.join(CWD, '.cpdctl.config.yml')

response = requests.get(CPDCTL_RELEASES_URL)
assets = response.json()[0]['assets']
platform_asset = next(a for a in assets if CPDCTL_ARCH in a['name'])
cpdctl_url = platform_asset['url']
cpdctl_file_name = platform_asset['name']

response = requests.get(cpdctl_url, headers={'Accept': 'application/octet-stream'})
with open(cpdctl_file_name, 'wb') as f:
    f.write(response.content)
    
display(HTML('<code>cpdctl</code> binary downloaded from: <a href="{}">{}</a>'.format(platform_asset['browser_download_url'], platform_asset['name'])))

In [3]:
%%capture

%env PATH={CWD}:{PATH}
%env CPD_CONFIG={CPD_CONFIG}

### 1.2 Check cpdctl version

In [4]:
if cpdctl_file_name.endswith('tar.gz'):
    with tarfile.open(cpdctl_file_name, "r:gz") as tar:
        tar.extractall()
elif cpdctl_file_name.endswith('zip'):
    with zipfile.ZipFile(cpdctl_file_name, 'r') as zf:
        zf.extractall()

if CPD_CONFIG and os.path.exists(CPD_CONFIG):
    os.remove(CPD_CONFIG)
    
version_r = ! cpdctl version
CPDCTL_VERSION = version_r.s

print("cpdctl version: {}".format(CPDCTL_VERSION))

cpdctl version: 1.1.132


### 1.3 Add CPD cluster configuration

In [11]:
CPD_USER_NAME = 'dhshi'
CPD_USER_PASSWORD = 'passw0rd'
CPD_URL = 'https://cpd-cpd-instance.apps.cp4d404ugi.cp.fyre.ibm.com'

Add "cpd406_user" user to the cpdctl configuration

In [12]:
! cpdctl config user set cpd_user --username {CPD_USER_NAME} --password {CPD_USER_PASSWORD}

Add "cpd406" cluster to the cpdctl configuration

In [13]:
! cpdctl config profile set cpd --url {CPD_URL}

Add "cpd406" context to the cpdctl configuration

In [14]:
! cpdctl config context set cpd --profile cpd --user cpd_user

List available contexts

In [15]:
! cpdctl config context list

Name                          Profile                       User                       Current   
inClusterEnvironmentContext   inClusterEnvironmentProfile   inClusterEnvironmentUser   *   


Make sure you use context which you prefer

In [18]:
! cpdctl config context use inClusterEnvironmentContext

Switched to context "inClusterEnvironmentContext".


List available projects in context

In [19]:
! cpdctl project list

...
ID                                     Name                                                Created                    Description                                          Tags   
0619c2d3-2b75-42f7-97c1-9d898fdef44c   Mortgage default project                            2022-02-11T12:02:58.633Z                                                        []   
19a29ada-3e9c-4147-bf35-4cf431ed3a26   AutoAI-TD-Sub                                       2022-01-25T01:20:57.078Z   The classification goal to train a model that can…   []   
2ba058a6-4ef7-4b3b-acde-44baaf288efb   test-analytics-project-git-ibm-sk-1                 2022-04-01T13:10:46.824Z                                                        []   
32d75b88-b113-4bd4-9502-cd2f471a8623   MLOps-CPD3.0-SWAT-StarterKit-AutoAI-Project-v0.96   2022-01-24T16:02:06.209Z                                                        []   
3ea4045c-4ff0-4995-8c80-849d1854b69f   julian                                              2022-03-30T14:18:2

## 2. Demo cpdctl connections commands

### 2.0 Create a project for connection or Choose an existing project

To create a connection, first you need to have a project, catalog or space.
You can either choose an existing project:

In [20]:
result = ! cpdctl project list --output json -j "(resources[].metadata.guid)[0]" --raw-output
PROJECT_ID = result.s
print(f'project id: {PROJECT_ID}')

# You can also specify your project id directly:
# PROJECT_ID = "Your project ID"

project id: 0619c2d3-2b75-42f7-97c1-9d898fdef44c


Or create a new project:

In [21]:
import uuid
STORAGE = {"type": "assetfiles", "guid": str(uuid.uuid4())}
STORAGE_JSON = json.dumps(STORAGE)
result = ! cpdctl project create --name "cpdctl-test-proj" --output json --raw-output --generator cli --storage '{STORAGE_JSON}' -j 'location'
PROJECT_ID = result.s.split('/')[-1]
print(f'project id: {PROJECT_ID}')

project id: 1bc0f8a6-6f33-439e-8299-025b2f4cb383


### 2.1 Create a connection

Here, as an example, connection properties to DB2 were used. Type your connection properties.

In [25]:
CREATE_CONN_PROPERTIES = '''
{
      "database": "AIOSDB2",
      "host": "9.30.248.108",
      "port": "50000",
      "password": "T3l3phon3passw0rd",
      "username": "db2inst1"
}
'''

In [26]:
result = ! cpdctl connection create --name "cpdctl-test-conn" --description "test connection to aiosdb2" --datasource-type "db2" --project-id '{PROJECT_ID}' --properties '{CREATE_CONN_PROPERTIES}' -j metadata.asset_id --origin-country us --output json -j 'metadata.asset_id'
CONNECTION_ID = result.s[1:-1]
print(f'connection id: {CONNECTION_ID}')

connection id: 5aa42f73-19ba-468a-881c-f09499bad7fe


### 2.2 List connections

In [27]:
! cpdctl connection list --project-id '{PROJECT_ID}'

...
ID                                     Name               Description                  Type         Tags   
5aa42f73-19ba-468a-881c-f09499bad7fe   cpdctl-test-conn   test connection to aiosdb2   connection   []   


In [28]:
! cpdctl connection list --project-id '{PROJECT_ID}' --entity-name "cpdctl-test-conn"

...
ID                                     Name               Description                  Type         Tags   
5aa42f73-19ba-468a-881c-f09499bad7fe   cpdctl-test-conn   test connection to aiosdb2   connection   []   


In [29]:
! cpdctl connection list --project-id '{PROJECT_ID}' --entity-flags "restricted" --sort "entity.name"

...
ID                                     Name               Description                  Type         Tags   
5aa42f73-19ba-468a-881c-f09499bad7fe   cpdctl-test-conn   test connection to aiosdb2   connection   []   


### 2.3 Discover connection ad-hoc

In [33]:
! cpdctl connection discover-adhoc --path "/DB2INST1" --datasource-type "db2" --name "conn_db2_adhoc" --properties '{CREATE_CONN_PROPERTIES}'

...
                 
next          <Nested Object>   
path          /DB2INST1   
prev          <Nested Object>   
total_count   1   
asset_types   <Array>   
assets        <Array>   
first         <Nested Object>   


In [35]:
! cpdctl connection discover-adhoc --path "/DB2INST1" --datasource-type "db2" --name "conn_db2_adhoc" --properties '{CREATE_CONN_PROPERTIES}' --output json

{
  "asset_types": [
    {
      "dataset": true,
      "dataset_container": false,
      "type": "table"
    }
  ],
  "assets": [
    {
      "id": "TEST",
      "name": "TEST",
      "path": "/DB2INST1/TEST",
      "type": "table"
    }
  ],
  "first": {
    "href": "https://internal-nginx-svc/v2/connections/assets?offset=0\u0026limit=100\u0026path=%2FDB2INST1"
  },
  "next": {
    "href": "https://internal-nginx-svc/v2/connections/assets?offset=100\u0026limit=100\u0026path=%2FDB2INST1"
  },
  "path": "/DB2INST1",
  "prev": {
    "href": "https://internal-nginx-svc/v2/connections/assets?offset=0\u0026limit=100\u0026path=%2FDB2INST1"
  },
  "total_count": 1
}


### 2.4 Discover data asset (not available until CPD 3.5)

To discover data asset, we first need to have a connected data asset ID. 

To get it we may create a new connected data asset or take an existing data asset ID.


#### Creating new connected data asset

Replace properties with your data. Especially connection_path.

In [39]:
ASSET_METADATA = '''
{
      "name": "connected-data-asset",
      "asset_type": "data_asset",
      "origin_country": "us",
      "asset_category": "USER"
}
'''
ASSET_ENTITY = '''
{
        "data_asset": {      
            "mime_type": "text/csv",
            "dataset": false
        }
}
'''
ATTACHMENTS = '''
[
        {
            "asset_type": "data_asset",
            "connection_id": "5aa42f73-19ba-468a-881c-f09499bad7fe",
            "connection_path": "DB2INST1/TEST"
        }
]
'''

ATTACHMENTS = ATTACHMENTS.replace("5aa42f73-19ba-468a-881c-f09499bad7fe", CONNECTION_ID)
result = ! cpdctl asset data-asset create --project-id '{PROJECT_ID}' --metadata '{ASSET_METADATA}' --entity '{ASSET_ENTITY}' --attachments '{ATTACHMENTS}' --output json -j metadata.asset_id
DATA_ASSET_ID = result.s[1:-1]
print(f'data asset id: {DATA_ASSET_ID}')

data asset id: 9044bbfc-2cd0-4f89-bb91-4400156cff73


#### Discover data asset

In [40]:
! cpdctl connection discover-data-asset --data-asset-id '{DATA_ASSET_ID}' --project-id '{PROJECT_ID}'

...
first             path            total_count   name   type   
<Nested Object>   DB2INST1/TEST   1             COL1   <Nested Object>   
<Nested Object>   DB2INST1/TEST   1             COL2   <Nested Object>   


In [41]:
! cpdctl connection discover-data-asset --data-asset-id '{DATA_ASSET_ID}' --project-id '{PROJECT_ID}' --output json

{
  "fields": [
    {
      "name": "COL1",
      "type": {
        "length": 10,
        "nullable": true,
        "scale": 0,
        "signed": true,
        "type": "integer"
      }
    },
    {
      "name": "COL2",
      "type": {
        "length": 10,
        "nullable": true,
        "scale": 0,
        "signed": true,
        "type": "integer"
      }
    }
  ],
  "first": {
    "href": "https://internal-nginx-svc/v2/connections/assets/9044bbfc-2cd0-4f89-bb91-4400156cff73?project_id=1bc0f8a6-6f33-439e-8299-025b2f4cb383\u0026userfs=false\u0026offset=0\u0026limit=100"
  },
  "path": "DB2INST1/TEST",
  "total_count": 1
}


### 2.5 Get connection

In [42]:
! cpdctl connection get --project-id '{PROJECT_ID}' --connection-id '{CONNECTION_ID}'

...
                  
ID:            5aa42f73-19ba-468a-881c-f09499bad7fe   
Name:          cpdctl-test-conn   
Description:   test connection to aiosdb2   
Type:          connection   
Tags:          []   


### 2.6 Update connection

Here is an example of updating name of connection, fom existing to the new one

In [43]:
UPDATE_CONN_PROPERTIES = '''
[{
      "op": "replace",
      "path": "/name",
      "from": "cpdctl-test-conn",
      "value": "updated-conn-name"
}]
'''

! cpdctl connection update --project-id '{PROJECT_ID}' --connection-id '{CONNECTION_ID}' --patch-request '{UPDATE_CONN_PROPERTIES}'

...
                  
Name:          updated-conn-name   
Description:   test connection to aiosdb2   
Tags:          []   


### 2.7 List actions

In [44]:
! cpdctl connection list-actions --project-id '{PROJECT_ID}' --connection-id '{CONNECTION_ID}'

...
Name               Description   
get_record_count   Get the number of rows in the specified table   


In [45]:
! cpdctl connection list-actions --project-id '{PROJECT_ID}' --connection-id '{CONNECTION_ID}' --output json

{
  "actions": [
    {
      "description": "Get the number of rows in the specified table",
      "name": "get_record_count",
      "properties": {
        "input": [
          {
            "description": "Name of the schema that contains the table",
            "hidden": false,
            "label": "Schema name",
            "masked": false,
            "multiline": false,
            "name": "schema_name",
            "readonly": false,
            "required": false,
            "type": "string",
            "user_defined": false
          },
          {
            "description": "Name of the table for which to obtain the number of rows",
            "hidden": false,
            "label": "Table name",
            "masked": false,
            "multiline": false,
            "name": "table_name",
            "readonly": false,
            "required": true,
            "type": "string",
            "user_defined": false
          }
        ],
        "output": [
          {
         

### 2.8 Perform action

Actions are different for each connector.
Action name with it's necassary properties can be taken from the list-actions output (look at 2.7 List actions)

In [46]:
PERFORM_ACTION_PROPS = '''
{
        "schema_name": "DB2INST1",
        "table_name": "TEST"
}
'''

! cpdctl connection perform-action --project-id '{PROJECT_ID}' --connection-id '{CONNECTION_ID}' --action-name "get_record_count" --request='{PERFORM_ACTION_PROPS}'

...
                  
record_count   3   


### 2.9 Discover connection

In [47]:
! cpdctl connection discover --project-id '{PROJECT_ID}' --connection-id '{CONNECTION_ID}' --path="/DB2INST1"

...
                 
total_count   1   
asset_types   <Array>   
assets        <Array>   
first         <Nested Object>   
path          /DB2INST1   


In [48]:
! cpdctl connection discover --project-id '{PROJECT_ID}' --connection-id '{CONNECTION_ID}' --path="/DB2INST1" --output json

{
  "asset_types": [
    {
      "dataset": true,
      "dataset_container": false,
      "type": "table"
    }
  ],
  "assets": [
    {
      "id": "TEST",
      "name": "TEST",
      "path": "/DB2INST1/TEST",
      "type": "table"
    }
  ],
  "first": {
    "href": "https://internal-nginx-svc/v2/connections/5aa42f73-19ba-468a-881c-f09499bad7fe/assets?project_id=1bc0f8a6-6f33-439e-8299-025b2f4cb383\u0026userfs=false\u0026offset=0\u0026limit=100\u0026path=%2FDB2INST1"
  },
  "path": "/DB2INST1",
  "total_count": 1
}


### 2.10 Delete connection

In [49]:
! cpdctl connection delete --project-id '{PROJECT_ID}' --connection-id '{CONNECTION_ID}'

...
OK


### 2.11 Clean up - Delete project created at the beggining (containing our connection)

In [50]:
! cpdctl project delete --project-id '{PROJECT_ID}'

...
OK


## 3. Demo cpdctl connection datasource types commands

### 3.1 List datasource types

In [51]:
! cpdctl connection datasource-type list

...
ID                                     Name                        Description                                          Type       Tags   
029e5d1c-ba73-4b09-b742-14c3a39b6cf9   informix                    IBM Informix database                                database   []   
048ed1bf-516c-46f0-ae90-fa3349d8bc1c   postgresql-ibmcloud         Databases for PostgreSQL database                    database   []   
05b7f0ea-6ae4-45e2-a455-cc280f110825   googlecloudstorage          Google Cloud Storage                                 file       []   
05c58384-862e-4597-b19a-c71ea7e760bc   impala                      Cloudera Impala database                             database   []   
06847b16-07b4-4415-a924-c63d11a17aa1   salesforce                  Salesforce.com                                       database   []   
0bd5946b-6fcb-4253-bf76-48b362d24a89   datastax-ibmcloud           DataStax on IBM Cloud database                       database   []   
0c431748-2572-11ea-978f-2e728ce8812

In [52]:
! cpdctl connection datasource-type list --sort entity.name --limit 5

...
ID                                     Name               Description                                          Type   Tags   
a0b1d14a-4767-404c-aac1-4ce0e62818c3   amazons3           Amazon Simple Storage Service (S3)                   file   []   
81bafdbd-b7c6-45c5-a4fd-6ec135f66f4e   assetfiles         Asset Files                                          file   []   
9a22e0af-8d19-4c4e-9aea-1d733e81315b   azureblobstorage   Microsoft Azure Blob Storage                         file   []   
6863060d-97c4-4653-abbe-958bde533f8c   azuredatalake      Microsoft Azure Data Lake Store via the WebHDFS A…   file   []   
2a7b4fa1-c770-4807-8871-a3c5def5aa2d   azurefilestorage   Microsoft Azure File Storage                         file   []   
Next token: 'https://internal-nginx-svc/v2/datasource_types?offset=5&limit=5&sort=entity.name'


In [53]:
! cpdctl connection datasource-type list --entity-environment "private" --entity-product "igc"

...
ID                                     Name               Description                       Type       Tags   
0fd83fe5-8995-4e2e-a1be-679bb8813a6d   hive               Apache Hive database              database   []   
48695e79-6279-474a-b539-342625d3dfc2   sqlserver          Microsoft SQL Server database     database   []   
6bcaf300-30b3-11eb-adc1-0242ac120002   sqlquery           IBM SQL Query                     database   []   
8b8fcd6d-8f95-49c7-8195-c72c95c9a84b   oracle-amazon      Amazon RDS for Oracle database    database   []   
8c1a4480-1c29-4b33-9086-9cb799d7b157   db2                IBM Db2 database                  database   []   
8e65204d-6156-49e7-96e5-d635b2aa05f6   mongodb-ibmcloud   Databases for MongoDB database    database   []   
971223d3-093e-4957-8af9-a83181ee9dd9   oracle             Oracle database                   database   []   
c10e5224-f17d-4524-844f-e97b1305e489   hdfs-apache        Apache HDFS via the WebHDFS API   file       []   
c6fb9293-51eb

In [54]:
! cpdctl connection datasource-type list --connection-properties true --interaction-properties true --actions true

...
ID                                     Name                        Description                                          Type       Tags   
029e5d1c-ba73-4b09-b742-14c3a39b6cf9   informix                    IBM Informix database                                database   []   
048ed1bf-516c-46f0-ae90-fa3349d8bc1c   postgresql-ibmcloud         Databases for PostgreSQL database                    database   []   
05b7f0ea-6ae4-45e2-a455-cc280f110825   googlecloudstorage          Google Cloud Storage                                 file       []   
05c58384-862e-4597-b19a-c71ea7e760bc   impala                      Cloudera Impala database                             database   []   
06847b16-07b4-4415-a924-c63d11a17aa1   salesforce                  Salesforce.com                                       database   []   
0bd5946b-6fcb-4253-bf76-48b362d24a89   datastax-ibmcloud           DataStax on IBM Cloud database                       database   []   
0c431748-2572-11ea-978f-2e728ce8812

### 3.2 Get datasource type

In [55]:
! cpdctl connection datasource-type get --datasource-type dashdb

...
                  
ID:            cfdcb449-1204-44ba-baa6-9a8a878e6aa7   
Name:          dashdb   
Description:   Db2 Warehouse   
Type:          database   
Tags:          []   


In [56]:
! cpdctl connection datasource-type get --datasource-type dashdb --connection-properties true --interaction-properties true

...
                  
ID:            cfdcb449-1204-44ba-baa6-9a8a878e6aa7   
Name:          dashdb   
Description:   Db2 Warehouse   
Type:          database   
Tags:          []   


In [57]:
! cpdctl connection datasource-type get --datasource-type db2

...
                  
ID:            8c1a4480-1c29-4b33-9086-9cb799d7b157   
Name:          db2   
Description:   IBM Db2 database   
Type:          database   
Tags:          []   


In [58]:
! cpdctl connection datasource-type get --datasource-type db2 --connection-properties true --interaction-properties true

...
                  
ID:            8c1a4480-1c29-4b33-9086-9cb799d7b157   
Name:          db2   
Description:   IBM Db2 database   
Type:          database   
Tags:          []   
